In [1]:
# ==============================================================================
# AUTHOR: Ana Barrio
# DATE: 5-02-26
# DESCRIPTION: Integrity check to validate topological consistency between 
#              Non-Tortuous and OutGeom graph versions.
# ==============================================================================

"""
GRAPH TOPOLOGY VALIDATOR

This script calculates the "Match Ratio" between two graph files to ensure 
that connectivity is preserved across different processing stages.

LOGIC:
1. It builds a reference set of all connections (node-id pairs) from the 
   Non-Tortuous graph.
2. It iterates through the OutGeom graph edges and verifies if the same 
   logical connections exist.
3. It outputs the total count and the ratio of preserved edges.
"""

import pickle

def inspect(p):
    with open(p,"rb") as f:
        d = pickle.load(f, encoding="latin1")
    G = d["graph"] if isinstance(d, dict) and "graph" in d else d
    print("\n===", p, "===")
    print("v attrs:", G.vs.attributes())
    print("e attrs:", G.es.attributes())

inspect("/home/admin/Ana/MicroBrain/18_igraph.pkl")
inspect("/home/admin/Ana/MicroBrain/output/graph_18_OutGeom.pkl")



=== /home/admin/Ana/MicroBrain/18_igraph.pkl ===
v attrs: ['id', 'coords', 'annotation', 'distance_to_surface', 'radii']
e attrs: ['connectivity', 'nkind', 'radius', 'diameter', 'length']

=== /home/admin/Ana/MicroBrain/output/graph_18_OutGeom.pkl ===
v attrs: ['id']
e attrs: ['nkind', 'radius', 'diameter', 'length', 'geom_start', 'geom_end', 'length_tortuous', 'tortuosity']


In [2]:

p_non="/home/admin/Ana/MicroBrain/18_igraph.pkl"
p_out="/home/admin/Ana/MicroBrain/output/graph_18_OutGeom.pkl"

non=pickle.load(open(p_non,"rb"),encoding="latin1")
G_non = non["graph"] if isinstance(non,dict) and "graph" in non else non

out=pickle.load(open(p_out,"rb"),encoding="latin1")
G_out = out["graph"] if isinstance(out,dict) else out

# set de edges del NON por (idmin,idmax)
non_keys=set()
for e in G_non.es:
    a,b = map(int, e["connectivity"])
    non_keys.add((a,b) if a<b else (b,a))

# contar coincidencias con OUTGEOM
match=0
for e in G_out.es:
    u,v = e.tuple
    idu=int(G_out.vs[u]["id"]); idv=int(G_out.vs[v]["id"])
    k=(idu,idv) if idu<idv else (idv,idu)
    if k in non_keys:
        match += 1

print("OUT edges:", G_out.ecount())
print("NON edges:", G_non.ecount())
print("Matches by ID-pair:", match)
print("Match ratio:", match / max(G_out.ecount(),1))


OUT edges: 4620894
NON edges: 4620894
Matches by ID-pair: 4620894
Match ratio: 1.0
